Importing the necessary libraries

In [19]:
import os
import numpy as np
import pandas as pd

Importing the coordinate data

In [20]:
data = pd.read_csv('Paris_Locations.csv')
stars_data = pd.read_excel('Book 4.xlsx', engine = 'openpyxl')
#pd.read_csv('Book 4.csv', encoding='utf-8')
#data
data_df = pd.DataFrame(data)
#data_df

Haversine Function

In [21]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points 
    on the Earth's surface using the Haversine formula with numpy.
    
    Parameters:
        lat1, lon1: Latitude and longitude of point 1 (in decimal degrees).
        lat2, lon2: Latitude and longitude of point 2 (in decimal degrees).
    
    Returns:
        Distance in kilometers.
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Radius of the Earth in kilometers (mean value)
    R = 6371.0
    return R * c

In [22]:
distances = []

for i, (place1, lat1, lon1) in data_df.iterrows():
    for j, (place2, lat2, lon2) in data_df.iterrows():
        if i != j:  # Skip distance from a place to itself
            distance = haversine(lat1, lon1, lat2, lon2)
            distances.append({
                "Place 1": place1,
                "Place 2": place2,
                "Distance (km)": distance
            })

distance_df = pd.DataFrame(distances)

In [23]:
#distances_df = distance_df.sort_values(by="Place 1").reset_index(drop=True)
distances_df_odd_rows = distance_df.iloc[1::2]
#distances_df

In [24]:
# Clean up column names and ensure proper matching
reference_data = stars_data.rename(columns={"Titel": "Place", "Stars": "Stars"})

# Merge distances_df_odd_rows with the reference data to add "stars" columns
distances_df_odd_rows = distances_df_odd_rows.merge(
    reference_data[["Place", "Stars"]], 
    left_on="Place 1", 
    right_on="Place", 
    how="left"
).rename(columns={"Stars": "Stars Place 1"}).drop(columns=["Place"])

distances_df_odd_rows = distances_df_odd_rows.merge(
    reference_data[["Place", "Stars"]], 
    left_on="Place 2", 
    right_on="Place", 
    how="left"
).rename(columns={"Stars": "Stars Place 2"}).drop(columns=["Place"])

In [25]:
# Sort the DataFrame by "Stars Place 1" and "Stars Place 2"
distances_df_sorted = distances_df_odd_rows.sort_values(
    by=["Stars Place 1", "Stars Place 2"],
    ascending=[False, False] 
)

# Group by "Place 1" to see the sorted grouping
#grouped_distances_df = distances_df_sorted.groupby("Place 1")


In [26]:
# Export the sorted DataFrame directly as a CSV
file_path = "Distances_Between_Places_Sorted.csv"
distances_df_sorted.to_csv(file_path, index=False)

# If you want to perform a groupby operation and export it
grouped_data = distances_df_sorted.groupby("Place 1").apply(lambda x: x).reset_index(drop=True)

# Export the grouped data as a CSV
file_path_grouped = "Distances_Between_Places_Grouped.csv"
grouped_data.to_csv(file_path_grouped, index=False)

C:\Users\Asmarani\AppData\Local\Temp\ipykernel_11080\213093926.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = distances_df_sorted.groupby("Place 1").apply(lambda x: x).reset_index(drop=True)
